In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import InceptionResNetV2

from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers, Sequential
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [ ]:
batch_size = 64

In [ ]:
# rescale normalizes the data for the images.
# Data is split into train and validation sets
image_data_gen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    validation_split=0.5)

train_it = image_data_gen.flow_from_directory(directory='/content/drive/MyDrive/ML_Project_Dataset/train',
                                              target_size=(299, 224),
                                              color_mode='rgb',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='training'
                                             )

val_it = image_data_gen.flow_from_directory(directory='/content/drive/MyDrive/ML_Project_Dataset/train',
                                              target_size=(299, 299),
                                              color_mode='rgb',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='validation'
                                             )

Found 8389 images belonging to 12 classes.
Found 8384 images belonging to 12 classes.


In [ ]:
dnn201 = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    input_tensor=None,
    input_shape=(224,224,3),
    weights = 'imagenet'
)

dnn201.trainable= False

global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
predictions = Dense(12, activation='softmax')

model = Sequential([
    dnn201,
    global_avg_layer,
    
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    predictions
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 5, 5, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1536)              0         
_________________________________________________________________
batch_normalization_1431 (Ba (None, 1536)              6144      
_________________________________________________________________
dropout_10 (Dropout)         (None, 1536)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              1573888   
_________________________________________________________________
dense_18 (Dense)             (None, 512)               524800    
_________________________________________________________________
batch_normalization_1432 (Ba (None, 512)              

In [ ]:
epochs = 2
model_hist = model.fit(train_it,
#                       validation_data=val_it)
                       epochs=epochs, )
model.save('InceptionResNetV2')

Epoch 1/2
132/132 [==============================] - 5563s 42s/step - loss: 2.4086 - accuracy: 0.3776
Epoch 2/2
 96/132 [====================>.........] - ETA: 13:18 - loss: 1.5023 - accuracy: 0.5519